In [462]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn import GRU
import numpy as np

In [463]:
users_df = pd.read_csv("../dataset/users.csv", sep=";")
artworks_df = pd.read_csv("../dataset/artworks.csv", sep=";")

In [464]:
comments = []
for idx, row in artworks_df.iterrows():
    for col in artworks_df.columns:
        if col.startswith("comment_") and not pd.isna(row[col]):
            comment = eval(row[col])
            for c in comment:
                comments.append(
                    {
                        "user_name": c["comment_author"],
                        "artwork_title": row["title"],
                        "timestamp": c["comment_date"],
                    }
                )

In [465]:
interactions_df = pd.DataFrame(comments)
user_map = {name: idx for idx, name in enumerate(users_df["name"].unique())}
artwork_map = {
    title: idx + len(user_map)
    for idx, title in enumerate(artworks_df["title"].unique())
}
interactions_df["timestamp"].replace("N/A", "01/01/1970", inplace=True)
interactions_df["timestamp"] = (
    pd.to_datetime(interactions_df["timestamp"], format="%d/%m/%Y")
)
interactions_df["timestamp"] = interactions_df["timestamp"].astype("int64") // 10**9
source_nodes = interactions_df["user_name"].map(user_map).tolist()
destination_nodes = interactions_df["artwork_title"].map(artwork_map).tolist()
timestamps = interactions_df["timestamp"].tolist()

C:\Users\LorenzoStancato\AppData\Local\Temp\ipykernel_6696\2488800568.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  interactions_df["timestamp"].replace("N/A", "01/01/1970", inplace=True)


In [466]:
num_nodes = len(user_map) + len(artwork_map)
memory_dim = 128  
embedding_dim = 128

In [467]:
node_memory = torch.zeros((num_nodes, memory_dim), dtype=torch.float)
node_embeddings = torch.zeros((num_nodes, embedding_dim), dtype=torch.float)
last_update = torch.zeros(num_nodes, dtype=torch.long)

In [468]:
num_heads = 4
attention_weights = torch.nn.Parameter(
    torch.randn(num_heads, embedding_dim , embedding_dim )
)
time_encoding_weights = torch.nn.Parameter(
    torch.randn(num_heads, embedding_dim)
)


def multi_head_attention(source_emb, neighbor_embs, delta_times):
    delta_times = delta_times.unsqueeze(1).repeat(1, num_heads, 1)
    time_encoding = torch.sin(delta_times.unsqueeze(2) * time_encoding_weights)
    attention_scores = []
    for head in range(num_heads):
        head_attention_scores = source_emb @ attention_weights[head]
        print(f"head_attention_scores: {head_attention_scores.shape}")
        print(
            "time_encoding[:, head, :].squeeze(1) shape:",
            time_encoding[:, head, :].squeeze(1).shape,
        )
        head_attention_scores += time_encoding[:, head, :]
        attention_scores.append(F.softmax(head_attention_scores, dim=1))
    aggregated_embedding = torch.cat(
        [
            torch.sum(attn.unsqueeze(2) * neighbor_embs, dim=1)
            for attn in attention_scores
        ],
        dim=1,
    )

    return aggregated_embedding

In [469]:
decay_rate = torch.nn.Parameter(torch.ones(1))
def handle_staleness(last_update, current_time, memory):
    time_diff = current_time - last_update
    decay_factor = torch.exp(-decay_rate * time_diff.unsqueeze(1).float())
    decayed_memory = memory * decay_factor
    return decayed_memory

In [470]:
gru = GRU(input_size=memory_dim + embedding_dim, hidden_size=memory_dim)
def update_memory_gru(message, memory):
    message = message.unsqueeze(0)  
    memory = memory.unsqueeze(0)  
    updated_memory, _ = gru(message, memory)
    return updated_memory.squeeze(0)

In [471]:
norm_weights = torch.nn.Parameter(torch.randn(embedding_dim))
def normalize_time(delta_times):
    normalized_time = delta_times.unsqueeze(1) * norm_weights
    return torch.exp(-normalized_time)

In [472]:
W_memory = torch.randn((memory_dim + embedding_dim, memory_dim), requires_grad=True)
b_memory = torch.zeros(memory_dim, requires_grad=True)

W_output = torch.randn((memory_dim, 1), requires_grad=True)
b_output = torch.zeros(1, requires_grad=True)

In [473]:
def clean_nodes(nodes):
    nodes = [-1 if np.isnan(x) else int(x) for x in nodes]
    return nodes

In [474]:
def process_batch(
    source_nodes,
    destination_nodes,
    timestamps,
    last_update,
    node_memory,
    current_time,
):
    source_nodes = clean_nodes(source_nodes)
    destination_nodes = clean_nodes(destination_nodes)
    valid_indices = [
        i for i, s in enumerate(source_nodes) if s != -1 and destination_nodes[i] != -1
    ]
    if len(valid_indices) == 0:
        return node_memory
    source_nodes = torch.tensor(
        [source_nodes[i] for i in valid_indices], dtype=torch.long
    )
    destination_nodes = torch.tensor(
        [destination_nodes[i] for i in valid_indices], dtype=torch.long
    )
    timestamps = torch.tensor([timestamps[i] for i in valid_indices], dtype=torch.long)
    source_memory = node_memory[source_nodes]
    destination_memory = node_memory[destination_nodes]
    delta_times = timestamps - last_update[source_nodes]
    source_memory = handle_staleness(
        last_update[source_nodes], current_time, source_memory
    )
    normalized_time = normalize_time(delta_times)
    aggregated_embedding = multi_head_attention(
        source_memory, destination_memory, normalized_time
    )
    updated_memory = torch.tanh(aggregated_embedding @ W_memory + b_memory)
    node_memory[source_nodes] = updated_memory

    return node_memory

In [475]:
optimizer = torch.optim.Adam(
    [
        W_memory,
        b_memory,
        W_output,
        b_output,
        decay_rate,
        norm_weights,
        attention_weights,
        time_encoding_weights,
    ],
    lr=0.001,
)

In [476]:
num_epochs = 5
batch_size = 32
for epoch in range(num_epochs):
    total_loss = 0.0
    current_time = timestamps[-1] 

    for i in range(0, len(source_nodes), batch_size):
        batch_source = source_nodes[i : i + batch_size]
        batch_dest = destination_nodes[i : i + batch_size]
        batch_times = timestamps[i : i + batch_size]
        node_memory = process_batch(
            batch_source,
            batch_dest,
            batch_times,
            last_update,
            node_memory,
            current_time,
        )
        src_emb = node_memory[batch_source]
        dst_emb = node_memory[batch_dest]
        score = (src_emb * dst_emb).sum(dim=1) @ W_output + b_output

        label = torch.ones(batch_size)
        loss = F.binary_cross_entropy_with_logits(score.squeeze(), label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(source_nodes)}")

head_attention_scores: torch.Size([3, 128])
time encoding: torch.Size([3, 4, 128])


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1

In [ ]:
def recommend_artworks(user_name, top_k=5):
    user_idx = user_map.get(user_name)
    if user_idx is None:
        print("Utente non trovato.")
        return []

    user_memory = node_memory[user_idx]
    artwork_indices = torch.arange(len(user_map), num_nodes)
    user_emb = user_memory.unsqueeze(0)
    artworks_memory = node_memory[artwork_indices]
    scores = (user_emb * artworks_memory).sum(dim=1) @ W_output + b_output
    top_scores, top_indices = torch.topk(scores, k=top_k)
    recommended_artworks = [
        list(artwork_map.keys())[i - len(user_map)] for i in top_indices.tolist()
    ]

    return recommended_artworks

In [ ]:
user_to_recommend = "exarobibliologist" 
recommendations = recommend_artworks(user_to_recommend)
print(f"Raccomandazioni per {user_to_recommend}: {recommendations}")